In [1]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import datetime
from datetime import date 
import time
import json
import csv
import numpy as np
import re
import lxml.html
import os
import tabula

In [2]:
wta_path = "https://wtafiles.wtatennis.com/pdf/rankings/All_YTD_Prize_Money.pdf"

In [3]:
atp_path = "https://www.protennislive.com/posting/ramr/current_prize.pdf"

In [4]:
wta_dfs = tabula.read_pdf(wta_path, stream=False, user_agent="Mozilla/5.0", pages='1-11')
# read_pdf returns list of DataFrames
print(len(wta_dfs))

11


In [6]:
print(wta_dfs[0])
#print(wta_dfs[2].columns)

    Standing                       Name  Nat    Singles  Doubles  Mixed  \
0          1           SABALENKA, ARYNA  NaN  2,239,021      NaN    NaN   
1          2              ZHENG, QINWEN  CHN  1,439,163      NaN    NaN   
2          3               SWIATEK, IGA  POL  1,375,308      NaN    NaN   
3          4              HUNTER, STORM  AUS    261,881  932,733    NaN   
4          5        SINIAKOVA, KATERINA  CZE    230,924  932,438    NaN   
5          6           PAOLINI, JASMINE  ITA    905,313   52,527    NaN   
6          7            RYBAKINA, ELENA  KAZ    886,535      NaN    NaN   
7          8                GAUFF, COCO  USA    779,389      NaN    NaN   
8          9           KALINSKAYA, ANNA  NaN    747,123   30,061    NaN   
9         10          OSTAPENKO, JELENA  LAT    545,042  193,587    NaN   
10        11         YASTREMSKA, DAYANA  UKR    717,024      NaN    NaN   
11        12           GARCIA, CAROLINE  FRA    556,015   73,284    NaN   
12        13             

In [7]:
print(wta_dfs[2].columns)

Index(['Standing', 'Unnamed: 0', 'Name', 'Nat', 'Singles', 'Doubles', 'Mixed',
       'Total'],
      dtype='object')


In [ ]:
## TO DO: concat AFTER changing capitalization under "name" 

In [8]:
#only keep the columns that are common to the two dfs
#wta_dfs = wta_dfs[['Name', , ]
women_df= pd.DataFrame([])

for wta_df in wta_dfs:
    if (len(wta_df.columns) == 8):
        wta_df.drop(columns=['Name'], inplace=True)
        wta_df.rename(columns={'Unnamed: 0': 'Name'}, inplace=True)
    wta_df.rename(columns={'Standing': 'Rank'}, inplace=True)   
    wta_df = wta_df[['Rank','Name', 'Total', 'Singles','Doubles' ]]
    women_df = pd.concat([women_df, wta_df])
    


In [9]:
print(women_df.columns)

Index(['Rank', 'Name', 'Total', 'Singles', 'Doubles'], dtype='object')


In [10]:
#previously: didn't have the INPLACE parameter 

In [11]:
women_df.reset_index(drop = True, inplace=True)

In [12]:
#Cutting off at index 499
women_df = women_df[women_df['Rank'] <= 500]

In [13]:
len(women_df)

500

In [18]:
#women_df.to_csv("files/2023-10-09_wta.csv")

In [19]:
#some columns have missing player names - pages == resolved thanks to RENAME 

In [14]:
atp_dfs = tabula.read_pdf(atp_path, stream=True, user_agent="Mozilla/5.0", pages='1-12', pandas_options={'header': None})

In [21]:
#print(atp_dfs[0].columns)

In [22]:
#print(atp_dfs[0])

In [15]:
men_df= pd.DataFrame([])

for atp_df in atp_dfs:
    atp_df.columns = ['Rank', 'Name', 'Total', 'Singles','Doubles', 'Career']
    atp_df = atp_df[['Rank','Name', 'Total', 'Singles','Doubles' ]]
    men_df = pd.concat([men_df, atp_df])

In [16]:
men_df['Total'] = men_df['Total'].str.strip('$')

In [17]:
men_df['Singles'] = men_df['Singles'].str.strip('$')

In [18]:
men_df['Doubles'] = men_df['Doubles'].str.strip('$')

In [27]:
#NOT WORKING: men_df["Rank"] = men_df["Rank"].astype(int)

In [28]:
#print(men_df.columns)

In [19]:
men_df.reset_index(drop = True, inplace=True)

In [30]:
#WE JUST LOST DJOKOVIC!!!!  --> resolved with correct tabula formula 

In [31]:
#NOT WORKING, either as int or as string 
#men_df = men_df[men_df['Rank'] <= 500]

In [20]:
#we used the index 
men_df = men_df[men_df.index <= 499]

In [21]:
len(men_df)

500

In [34]:
#mkdir files 
#cd files 
#name each file with the date  
#also store all weekly rank data in a single file, for atp and wta ("all_weeks")

In [35]:
#already done:
#os.mkdir("files")

In [22]:
date = date.today()

In [23]:
print(date)

2024-03-18


In [38]:
women_df.to_csv(f"files/{date}_wta.csv")

In [39]:
#PROBLEM: the index vaues repeat themselves instead of being sequential == resolved via "inplace=true
#LETS SEE AFTER WE COMBINE 

In [40]:
men_df.to_csv(f"files/{date}_atp.csv")

In [24]:
men_df['gender'] = 'male'
women_df['gender'] = 'female'

In [42]:
mixed_df = pd.concat([women_df, men_df])

In [43]:
#print(mixed_df)

In [44]:
#print(mixed_df.iloc[55])

In [45]:
#print(mixed_df.iloc[552])

In [46]:
#problems with women_df from last session: 
# - ranks not in sequence: 
# - no names for some pages: 
# - doubles earnings missing last 3 digits:

In [47]:
#mixed_df.reset_index(inplace=True)
mixed_df.reset_index(drop = True, inplace=True)

In [48]:
#******** RETURN TO THIS AFTER 

In [49]:
#mixed_df.to_csv(f"files/{date}_mixed.csv")

In [50]:
#HOW/WHAT do we compare??
#compare [gender]=women at RANK i vs gender - men at rank i
#Do we keep the names or just the ranks? 
# what does DW show? 
#OPTION 1
#weekly updated table of the difference between W1 M1
#BUT if you hover over any of the rows, you get the info on absolute earnings and player name ()both are bound to change due to ranking change) 
#OPTION 2
#Show difference at each rank, plus respective names??? CTA -- hover to see actual earnings$$ 

#create a new df called "difference" loop thru mixed df by looping with gender-female from rank 1 to 500, 
# then gender-male at rank value 1 to 500, 


In [51]:
# for player in mixed_df:
#     if mixed_df[[gender = female && index ]]

In [52]:
# diff = 0
# for woman in women_df: 
#     difference_df = pd.DataFrame()


    
    

In [53]:
# ranks = []
# diff = []

# for index in women_df.index:
#     men_total = men_df.iloc[index]['Total']
#     women_total = women_df.iloc[index]['Total']
#     ranks.append(index + 1)
#     diff.append(men_total - women_total)
    

In [54]:
#remove commas 

In [55]:
# men_total = int(men_df.iloc[index]['Total'].replace(',', ''))
# women_total = int(women_df.iloc[index]['Total'].replace(',', ''))

In [56]:
# ranks = []
# diff = []

# for index in women_df.index:
#     men_total = int(men_df.iloc[index]['Total'].replace(',', ''))
#     women_total = int(women_df.iloc[index]['Total'].replace(',', ''))
#     ranks.append(index + 1)
#     diff.append(men_total - women_total)

In [57]:
# len(diff)

In [58]:
# diff_df = pd.DataFrame([])
# diff_df['Rank'] = ranks
# diff_df['diff'] = diff

In [59]:
# diff_df.head()

In [60]:
#next: add names?? 

In [61]:
# diff_df = pd.DataFrame([])
# diff_df['Rank'] = ranks
# diff_df['diff'] = diff
# diff_df['women'] = women_df['Name']
# diff_df['men'] = men_df['Name']

In [62]:
# diff_df.head(50)

In [63]:
#NEXT: percentage difference 

In [64]:
#THIS CELL KEEPS ERRORING OUT: 
#
# ranks = []
# diff = []
# pct = []

# for index in women_df.index:
#     men_total = int(men_df.iloc[index]['Total'].replace(',', ''))
#     women_total = int(women_df.iloc[index]['Total'].replace(',', ''))
#     ranks.append(index + 1)
#     diff.append(men_total - women_total)
#     pct.append((women_total/men_total)*100)

In [25]:
#NEW CHAPTGPT:
# Make sure the indices in both dataframes match

ranks = []
diff = []
pct = []

# Iterate over the range of the smaller DataFrame to avoid IndexError
for index in range(min(len(women_df), len(men_df))):
    try:
        # Remove commas from 'Total' and convert to integer
        men_total = int(men_df.iloc[index]['Total'].replace(',', ''))
        women_total = int(women_df.iloc[index]['Total'].replace(',', ''))
        
        # Append the calculated values to the lists
        ranks.append(index + 1)
        diff.append(men_total - women_total)
        pct.append((women_total / men_total) * 100)
        
    except Exception as e:
        # Handle exceptions for each row and continue with the next
        print(f"Error processing row {index}: {e}")
        continue








In [26]:
diff_df = pd.DataFrame([])
diff_df['Rank'] = ranks
diff_df['diff'] = diff
diff_df['pct'] = pct
diff_df['women'] = women_df['Name']
diff_df['men'] = men_df['Name']

In [27]:
diff_df.head(20)

,Rank,diff,pct,women,men
0,1,297267,88.279446,"SABALENKA, ARYNA","Sinner, Jannik"
1,2,26787,98.172721,"ZHENG, QINWEN","Zverev, Alexander"
2,3,-64391,104.911905,"SWIATEK, IGA","Medvedev, Daniil"
3,4,69178,94.526156,"HUNTER, STORM","de Minaur, Alex"
4,5,-207697,121.733243,"SINIAKOVA, KATERINA","Djokovic, Novak"
5,6,-15842,101.681745,"PAOLINI, JASMINE","Hurkacz, Hubert"
6,7,25076,97.249265,"RYBAKINA, ELENA","Humbert, Ugo"
7,8,64913,92.311637,"GAUFF, COCO","Ruud, Casper"
8,9,-2903,100.374928,"KALINSKAYA, ANNA","Rublev, Andrey"
9,10,30841,95.991917,"OSTAPENKO, JELENA","Fritz, Taylor"


In [67]:
#diff_df.tail(30)

In [68]:
#SAVE THIS DIFF dataframe! --> feed to DW 

In [28]:
diff_df.to_csv(f"files/{date}_diff.csv")

In [70]:
#ALSO SAVE TO UNDATED csv!!! 

In [29]:
diff_df.to_csv("latest_diff.csv")

In [30]:
ls 

OCT9.ipynb                old_atp_live_ranking.csv  tennisOCT3.ipynb
README.md                 old_tennispay.ipynb       tennisgpt.ipynb
atp_live_ranking.csv      old_tennisweekly.ipynb    weekly_race_to_top.csv
files/                    pay_gap.ipynb
latest_diff.csv           pay_gap_old.ipynb


In [31]:
ls-la

total 6448
drwxr-xr-x  19 mfhan  staff      608 Mar 18 17:27 ./
drwxr-xr-x  12 mfhan  staff      384 Mar  8 11:26 ../
-rw-r--r--@  1 mfhan  staff    14340 Nov 20 20:42 .DS_Store
drwxr-xr-x  16 mfhan  staff      512 Mar 11 15:09 .git/
drwxr-xr-x   4 mfhan  staff      128 Oct 22 22:20 .github/
drwxr-xr-x  13 mfhan  staff      416 Mar 18 17:14 .ipynb_checkpoints/
-rw-r--r--   1 mfhan  staff    87443 Oct  9 19:05 OCT9.ipynb
-rw-r--r--   1 mfhan  staff       20 Oct 22 20:26 README.md
-rw-r--r--@  1 mfhan  staff       47 Sep 21 17:11 atp_live_ranking.csv
drwxr-xr-x  26 mfhan  staff      832 Mar 18 17:28 files/
-rw-r--r--@  1 mfhan  staff    34131 Mar 18 17:28 latest_diff.csv
-rw-r--r--@  1 mfhan  staff       20 Sep 21 17:03 old_atp_live_ranking.csv
-rw-r--r--   1 mfhan  staff    11970 Oct  9 14:40 old_tennispay.ipynb
-rw-r--r--   1 mfhan  staff  1565449 Oct  3 16:06 old_tennisweekly.ipynb
-rw-r--r--   1 mfhan  staff    29644 Mar 18 17:27 pay_gap.ipynb
-rw-r--r--   1 mfhan  staff    28835 Mar